## CODE

Below contain the code for our experiments.

### Preprocessing

In [ ]:
def trim_filename(file_name):
    # This regular expression matches 'Case' followed by any characters until '.mat'
    match = re.search(r'(Case.*?)(?=\.mat)', file_name)
    if match:
        return match.group(0)
    else:
        return 'No match found'

# Example usage
file_name = 'data/TF_204n_Case1_Baseline_RoofOnly_021perc_000deg_Test06.mat'
trimmed_name = trim_filename(file_name)
print(trimmed_name)

#### Imports

In [ ]:
import os
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.io import loadmat
import mat73
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.io import loadmat, savemat
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ConstantKernel as C
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import UndefinedMetricWarning, NotFittedError
import matlab.engine

#### Defining the Dataframes

In [ ]:
file_name_0 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_000deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_0)
df_0 = pd.DataFrame(annots['CpRoof'])
print(df_0.shape)

file_name_15 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_015deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_15)
df_15 = pd.DataFrame(annots['CpRoof'])
print(df_15.shape)

file_name_30 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_030deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_30)
df_30 = pd.DataFrame(annots['CpRoof'])
print(df_30.shape)

file_name_45 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_045deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_45)
df_45 = pd.DataFrame(annots['CpRoof'])
print(df_45.shape)

file_name_60 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_060deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_60)
df_60 = pd.DataFrame(annots['CpRoof'])
print(df_60.shape)

file_name_75 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_075deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_75)
df_75 = pd.DataFrame(annots['CpRoof'])
print(df_75.shape)

file_name_90 = 'data/TF_204n_Case1_Baseline_RoofOnly_044_5perc_090deg_Test05_Roof_Cp3sec.mat'
annots = mat73.loadmat(file_name_90)
df_90 = pd.DataFrame(annots['CpRoof'])
print(df_90.shape)

In [ ]:
# Start MATLAB engine
eng = matlab.engine.start_matlab()

used_blocks_number = 1
total_block_number = 2

def compute_min_peaks(df, num_taps, eng):
    results = [ None for _ in range(num_taps) ]

    # Iterate through each tap
    for tap in range(num_taps):
        y_true_1 = df.iloc[ : int(36279/2), tap]
        data_dict_1 = {'CP': y_true_1.to_numpy().reshape(-1, 1)}
        savemat('output_file_1.mat', data_dict_1)
        Cp_min_1, Cp_mean_1, Cp_std_1 = eng.Peaks('output_file_1.mat', nargout=3)
        y_true_2 = df.iloc[ int(36279/2):, tap]
        data_dict_2 = {'CP': y_true_2.to_numpy().reshape(-1, 1)}
        savemat('output_file_2.mat', data_dict_2)
        Cp_min_2, Cp_mean_2, Cp_std_2 = eng.Peaks('output_file_2.mat', nargout=3)
        
        results[tap] = ((Cp_min_1, Cp_mean_1, Cp_std_1), (Cp_min_2, Cp_mean_2, Cp_std_2))

    # Convert the results array to a DataFrame
    df_min_peaks = pd.DataFrame(results).transpose()
    return df_min_peaks

# Parameters

num_taps = 150  # Assuming there are 150 taps

# Compute min peaks for each DataFrame
df_stats_0 = compute_min_peaks(df_0, num_taps, eng)
df_stats_15 = compute_min_peaks(df_15, num_taps, eng)
df_stats_30 = compute_min_peaks(df_30, num_taps, eng)
df_stats_45 = compute_min_peaks(df_45, num_taps, eng)
df_stats_60 = compute_min_peaks(df_60, num_taps, eng)
df_stats_75 = compute_min_peaks(df_75, num_taps, eng)
df_stats_90 = compute_min_peaks(df_90, num_taps, eng)

df_stats_0.to_pickle('df_entire_double_stats_0.pkl')
df_stats_15.to_pickle('df_entire_double_stats_15.pkl')
df_stats_30.to_pickle('df_entire_double_stats_30.pkl')
df_stats_45.to_pickle('df_entire_double_stats_45.pkl')
df_stats_60.to_pickle('df_entire_double_stats_60.pkl')
df_stats_75.to_pickle('df_entire_double_stats_75.pkl')
df_stats_90.to_pickle('df_entire_double_stats_90.pkl')


In [ ]:
import numpy as np
import pandas as pd
from scipy.io import savemat
import matlab.engine

# Start MATLAB engine
eng = matlab.engine.start_matlab()
used_blocks_number = 1
total_block_number = 1

def compute_min_peaks(df, num_taps, eng):
    results = [ None for _ in range(num_taps) ]

    # Iterate through each tap
    for tap in range(num_taps):
        y_true = df.iloc[ : , tap]
        data_dict = {'CP': y_true.to_numpy().reshape(-1, 1)}
        savemat('output_file.mat', data_dict)
        Cp_min, Cp_mean, Cp_std = eng.Peaks('output_file.mat', nargout=3)
        results[tap] = (Cp_min, Cp_mean, Cp_std)

    # Convert the results array to a DataFrame
    df_min_peaks = pd.DataFrame(results).transpose()
    return df_min_peaks

# Parameters

num_taps = 150  # Assuming there are 150 taps

# Compute min peaks for each DataFrame
df_stats_0 = compute_min_peaks(df_0, num_taps, eng)
df_stats_15 = compute_min_peaks(df_15, num_taps, eng)
df_stats_30 = compute_min_peaks(df_30, num_taps, eng)
df_stats_45 = compute_min_peaks(df_45, num_taps, eng)
df_stats_60 = compute_min_peaks(df_60, num_taps, eng)
df_stats_75 = compute_min_peaks(df_75, num_taps, eng)
df_stats_90 = compute_min_peaks(df_90, num_taps, eng)

df_stats_0.to_pickle('df_entire_stats_0.pkl')
df_stats_15.to_pickle('df_entire_stats_15.pkl')
df_stats_30.to_pickle('df_entire_stats_30.pkl')
df_stats_45.to_pickle('df_entire_stats_45.pkl')
df_stats_60.to_pickle('df_entire_stats_60.pkl')
df_stats_75.to_pickle('df_entire_stats_75.pkl')
df_stats_90.to_pickle('df_entire_stats_90.pkl')

In [ ]:
first = ''

df_entire_0 = pd.read_pickle(first + 'df_entire_stats_0.pkl')
print(df_entire_0.shape)

df_entire_15 = pd.read_pickle(first + 'df_entire_stats_15.pkl')
print(df_entire_15.shape)

df_entire_30 = pd.read_pickle(first + 'df_entire_stats_30.pkl')
print(df_entire_30.shape)

df_entire_45 = pd.read_pickle(first + 'df_entire_stats_45.pkl')
print(df_entire_45.shape)

df_entire_60 = pd.read_pickle(first + 'df_entire_stats_60.pkl')
print(df_entire_60.shape)

df_entire_75 = pd.read_pickle(first + 'df_entire_stats_75.pkl')
print(df_entire_75.shape)

df_entire_90 = pd.read_pickle(first + 'df_entire_stats_90.pkl')
print(df_entire_90.shape)

df_entire_double_0 = pd.read_pickle(first + 'df_entire_double_stats_0.pkl')
print(df_entire_0.shape)

df_entire_double_15 = pd.read_pickle(first + 'df_entire_double_stats_15.pkl')
print(df_entire_15.shape)

df_entire_double_30 = pd.read_pickle(first + 'df_entire_double_stats_30.pkl')
print(df_entire_30.shape)

df_entire_double_45 = pd.read_pickle(first + 'df_entire_double_stats_45.pkl')
print(df_entire_45.shape)

df_entire_double_60 = pd.read_pickle(first + 'df_entire_double_stats_60.pkl')
print(df_entire_60.shape)

df_entire_double_75 = pd.read_pickle(first + 'df_entire_double_stats_75.pkl')
print(df_entire_75.shape)

df_entire_double_90 = pd.read_pickle(first + 'df_entire_double_stats_90.pkl')
print(df_entire_90.shape)


#### Locations of Pressure Taps

In [ ]:
# Horizontal coordinates (from left to right as they appear to increase):
horizontal_distances = [2.00, 5.20, 8.40, 12.00, 15.60, 20.11, 25.73, 31.15, 36.77, 42.19]

# Vertical coordinates (from bottom to top as they appear to increase):
vertical_distances = [2.00, 5.20, 8.15, 11.11, 15.60, 20.55, 25.28, 30.58, 34.72, 39.45, 44.40, 48.89, 51.85, 54.80, 58.00]

roof_pressure_tap_coordinates = [(x, y) for x in horizontal_distances for y in vertical_distances]

# Horizontal coordinates (from left to right as they appear to increase):
horizontal_distances = [0.5, 3.78, 7.06, 10.34]

# Vertical coordinates (from top to bottom as they appear to increase):
vertical_distances = [6.0, 3.25, 0.5]

each_pv_pressure_tap_coordinates = [(x, y) for y in vertical_distances for x in horizontal_distances]

pv_pressure_tap_coordinates = []

# add (44.90, 8.0) to the coordinates for each pv pressure tap
pv_pressure_tap_coordinates.append([(np.round(x + 44.90 + 0.20, 4), y + 8.0) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 55.94 + 0.20, 4), y + 8.0) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 66.98 + 0.20,4), y + 8.0) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 55.94 + 0.20, 4), y + 17.45) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 66.98 + 0.20, 4), y + 17.45) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 66.98 + 0.20, 4), y + 26.89) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 66.98 + 0.20, 4), y + 39.34) for x, y in each_pv_pressure_tap_coordinates])
pv_pressure_tap_coordinates.append([(np.round(x + 66.98 + 0.20, 4), y + 45.79) for x, y in each_pv_pressure_tap_coordinates])

# take arccos of 6.21 / 6.50 to find the angle
pv_angle_radian = np.arccos(6.21 / 6.50)
pv_angle_degree = np.degrees(pv_angle_radian)
pv_pressure_tap_coordinates[0]

# Let's create a function to duplicate and concatenate the lists
def create_combined_list(pv_panel_lists):
    combined_list = []
    for panel_list in pv_panel_lists:
        for coordinate in panel_list:
            # Add the coordinate twice, once for top (blue dot) and once for bottom (red dot)
            combined_list.append(coordinate)  # For the blue dot
        for coordinate in panel_list:
            # Add the coordinate twice, once for top (blue dot) and once for bottom (red dot)
            combined_list.append(coordinate)  # For the blue dot
    return combined_list

# Now we create the combined list
enumerated_pv_pressure_tap_coordinates = create_combined_list(pv_pressure_tap_coordinates)

# attach pv_pressure_tap_coordinates and combined list
pressure_tap_coordinates = roof_pressure_tap_coordinates + enumerated_pv_pressure_tap_coordinates

#### Predicting the Statistics of a Completely Unknown Time History

In [ ]:
# Predictions of all different models for a given wind angle and predicted tap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.model_selection._search')
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.model_selection._validation')

# Data preparation
taps_for_training = [65, 66, 67, 80, 82, 95, 96, 97]
tap_index = 81

taps_for_training = [46, 47, 48, 62, 63, 76, 77, 78]
tap_index = 61

taps_for_training = [2, 16, 17, 3, 18, 31, 32, 33]
tap_index = 1

df_entire = df_entire_0

# Extract data for the specified taps (index -1 because it's zero-indexed)
train_data = pd.DataFrame({
    'tap': taps_for_training,
    'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
    'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
    'min_peak': [df_entire.iloc[0, tap - 1] for tap in taps_for_training],
    'mean': [df_entire.iloc[1, tap - 1] for tap in taps_for_training],
    'std': [df_entire.iloc[2, tap - 1] for tap in taps_for_training]
})

# Extract data for tap 1 (index 0 because it's zero-indexed)
test_data = pd.DataFrame({
    'tap': [tap_index],
    'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
    'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
    'min_peak': [df_entire.iloc[0, tap_index-1]],
    'mean': [df_entire.iloc[1, tap_index-1]],
    'std': [df_entire.iloc[2, tap_index-1]]
})

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
X_test_scaled = scaler.transform(test_data[['x', 'y']])


# Initialize models
models = {
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNeighbors': KNeighborsRegressor(),
    'NeuralNetwork': MLPRegressor(),
    'GaussianProcess': GaussianProcessRegressor()
}

# Initialize dictionaries to store predictions, errors, and best parameters
best_predictions = {}
best_errors = {}
best_model_instance = {}
best_params = {}
model_errors = {model_name: {} for model_name in models}

# Perform regression for each statistic and each model
for stat in ['min_peak', 'mean', 'std']:
    y_train = train_data[stat]
    y_true = test_data[stat].values[0]

    min_error = float('inf')
    best_pred = None
    best_model_name = None
    best_model = None
    best_param = None

    for model_name, model in models.items():
        # Perform grid search with cross-validation
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3)
        grid_search.fit(X_train_scaled, y_train)

        print(f'Finished training {model_name} for {stat}')

        # Use the best model found
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)[0]

        percentage_error = 100 * abs((y_true - y_pred) / y_true)
        model_errors[model_name][stat] = percentage_error

        # Store the best parameters and errors for each model
        if model_name not in best_params:
            best_params[model_name] = {}
        best_params[model_name][stat] = {
            'params': grid_search.best_params_,
            'error': percentage_error,
            'pred': y_pred
        }

        if percentage_error < min_error:
            min_error = percentage_error
            best_pred = y_pred
            best_model_name = model_name
            best_param = grid_search.best_params_

    best_predictions[stat] = best_pred
    best_errors[stat] = min_error
    best_model_instance[stat] = models[best_model_name]

# Calculate and display percentage errors
for stat in ['min_peak', 'mean', 'std']:
    print(f'{stat} - True value: {test_data[stat].values[0]}, Predicted value: {best_predictions[stat]}, Percentage error: {best_errors[stat]:.2f}%, Best model: {best_model_instance[stat]}')

# Displaying the results in a tabular format
results_df = pd.DataFrame({
    'Statistic': ['min_peak', 'mean', 'std'],
    'True Value': [test_data[stat].values[0] for stat in ['min_peak', 'mean', 'std']],
    'Predicted Value': [best_predictions[stat] for stat in ['min_peak', 'mean', 'std']],
    'Percentage Error': [best_errors[stat] for stat in ['min_peak', 'mean', 'std']],
    'Best Model': [type(best_model_instance[stat]).__name__ for stat in ['min_peak', 'mean', 'std']]
})

# Display the DataFrame using Pandas
print(results_df)

# Print the best parameters and errors for each model
for model_name in best_params:
    print(f'\nModel: {model_name}')
    for stat in ['min_peak', 'mean', 'std']:
        print(f'  Statistic: {stat}')
        print(f'    Best Params: {best_params[model_name][stat]["params"]}')
        print(f'    Predicted Value: {best_params[model_name][stat]["pred"]}')
        print(f'    Percentage Error: {best_params[model_name][stat]["error"]:.2f}%')

# Create a table showing the percentage error of each statistic for each model
error_table = pd.DataFrame(model_errors).T
print("\nPercentage Errors for Each Model and Statistic:")
print(error_table)

In [ ]:
# Predictions for a fixed wind angle and predicted tap

taps_for_training = [65, 66, 67, 80, 82, 95, 96, 97]
tap_index = 81

taps_for_training = [46, 47, 48, 62, 63, 76, 77, 78]
tap_index = 61

taps_for_training = [2, 16, 17, 3, 18, 31, 32, 33]
tap_index = 1

df_entire = df_entire_0

# Extract data for the specified taps (index -1 because it's zero-indexed)
train_data = pd.DataFrame({
    'tap': taps_for_training,
    'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
    'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
    'min_peak': [df_entire.iloc[0, tap - 1] for tap in taps_for_training],
    'mean': [df_entire.iloc[1, tap - 1] for tap in taps_for_training],
    'std': [df_entire.iloc[2, tap - 1] for tap in taps_for_training]
})

# Extract data for tap 62 (index 61 because it's zero-indexed)
test_data = pd.DataFrame({
    'tap': [tap_index],
    'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
    'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
    'min_peak': [df_entire.iloc[0, tap_index-1]],
    'mean': [df_entire.iloc[1, tap_index-1]],
    'std': [df_entire.iloc[2, tap_index-1]]
})

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
X_test_scaled = scaler.transform(test_data[['x', 'y']])


# Define parameter grids for each model
param_grids = {
    'DecisionTree': {'max_depth': [None, 5, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'criterion': ['squared_error', 'friedman_mse', 'absolute_error']},
    'RandomForest': {'n_estimators': [25, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10, 20]},
    'GradientBoosting': {'n_estimators': [10, 25, 50, 100, 150], 'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2], 'max_depth': [2, 3, 5, 10]},
    'KNeighbors': {'n_neighbors': [2, 3, 5, len(taps_for_training)], 'weights': ['uniform', 'distance'], 'p': [1, 2]},
    'NeuralNetwork': {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)], 'alpha': [0.0001, 0.001, 0.01], 'solver': ['adam', 'lbfgs'], 'max_iter': [3000]},
    'GaussianProcess': {
        'kernel': [
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RationalQuadratic(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
        ],
        'alpha': [1e-2, 1e-5, 1e-10, 1e-12],
        'normalize_y': [False, True]
    }
}

# Initialize models
models = {
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNeighbors': KNeighborsRegressor(),
    'NeuralNetwork': MLPRegressor(),
    'GaussianProcess': GaussianProcessRegressor()
}


# Initialize dictionaries to store predictions, errors, and best parameters
best_predictions = {}
best_errors = {}
best_model_instance = {}
best_params = {}

# Perform regression for each statistic and each model
for stat in ['min_peak', 'mean', 'std']:
    y_train = train_data[stat]
    y_true = test_data[stat].values[0]

    min_error = float('inf')
    best_pred = None
    best_model_name = None
    best_model = None
    best_param = None

    for model_name, model in models.items():
        # Perform grid search with cross-validation
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3)
        grid_search.fit(X_train_scaled, y_train)

        # Use the best model found
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)[0]

        percentage_error = 100 * abs((y_true - y_pred) / y_true)

        # Store the best parameters and errors for each model
        if model_name not in best_params:
            best_params[model_name] = {}
        best_params[model_name][stat] = {
            'params': grid_search.best_params_,
            'error': percentage_error,
            'pred': y_pred
        }

        if percentage_error < min_error:
            min_error = percentage_error
            best_pred = y_pred
            best_model_name = model_name
            best_param = grid_search.best_params_

    best_predictions[stat] = best_pred
    best_errors[stat] = min_error
    best_model_instance[stat] = models[best_model_name]

# Calculate and display percentage errors
for stat in ['min_peak', 'mean', 'std']:
    print(f'{stat} - True value: {test_data[stat].values[0]}, Predicted value: {best_predictions[stat]}, Percentage error: {best_errors[stat]:.2f}%, Best model: {best_model_instance[stat]}')

# Displaying the results in a tabular format
results_df = pd.DataFrame({
    'Statistic': ['min_peak', 'mean', 'std'],
    'True Value': [test_data[stat].values[0] for stat in ['min_peak', 'mean', 'std']],
    'Predicted Value': [best_predictions[stat] for stat in ['min_peak', 'mean', 'std']],
    'Percentage Error': [best_errors[stat] for stat in ['min_peak', 'mean', 'std']],
    'Best Model': [type(best_model_instance[stat]).__name__ for stat in ['min_peak', 'mean', 'std']]
})

# Display the DataFrame using Pandas
print(results_df)

# Print the best parameters and errors for each model
for model_name in best_params:
    print(f'\nModel: {model_name}')
    for stat in ['min_peak', 'mean', 'std']:
        print(f'  Statistic: {stat}')
        print(f'    Best Params: {best_params[model_name][stat]["params"]}')
        print(f'    Predicted Value: {best_params[model_name][stat]["pred"]}')
        print(f'    Percentage Error: {best_params[model_name][stat]["error"]:.2f}%')


In [ ]:
# Predictions with a fixed predicted tap for different wind angles with the name and hyperparameters of the best performing model in each case

# angles list
angles = [0, 15, 30, 45, 60, 75, 90]

# Function to extract train and test data
def extract_data(df_entire, taps_for_training, tap_index):
    train_data = pd.DataFrame({
        'tap': taps_for_training,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
        'min_peak': [df_entire.iloc[0, tap - 1] for tap in taps_for_training],
        'mean': [df_entire.iloc[1, tap - 1] for tap in taps_for_training],
        'std': [df_entire.iloc[2, tap - 1] for tap in taps_for_training]
    })

    test_data = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'min_peak': [df_entire.iloc[0, tap_index-1]],
        'mean': [df_entire.iloc[1, tap_index-1]],
        'std': [df_entire.iloc[2, tap_index-1]]
    })

    return train_data, test_data

# Define parameter grids for each model
param_grids = {
    'DecisionTree': {'max_depth': [None, 5, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'criterion': ['squared_error', 'friedman_mse', 'absolute_error']},
    'RandomForest': {'n_estimators': [25, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10, 20]},
    'GradientBoosting': {'n_estimators': [10, 25, 50, 100, 150], 'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2], 'max_depth': [2, 3, 5, 10]},
    'KNeighbors': {'n_neighbors': [2, 3, 5, len(taps_for_training)], 'weights': ['uniform', 'distance'], 'p': [1, 2]},
    'NeuralNetwork': {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)], 'alpha': [0.0001, 0.001, 0.01], 'solver': ['adam', 'lbfgs'], 'max_iter': [3000]},
    'GaussianProcess': {
        'kernel': [
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RationalQuadratic(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
        ],
        'alpha': [1e-2, 1e-5, 1e-10, 1e-12],
        'normalize_y': [False, True]
    }
}


models = {
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNeighbors': KNeighborsRegressor(),
    'NeuralNetwork': MLPRegressor(),
    'GaussianProcess': GaussianProcessRegressor()
}



# Dictionary to store results
angle_results = {angle: {} for angle in angles}

# Loop through each angle
for angle in angles:
    print(f'Processing angle: {angle}')
    # Update df_entire based on the angle
    df_entire = eval(f'df_entire_{angle}')

    # Example taps for training and testing; you need to update these as per your requirement

    taps_for_training = [2, 16, 17, 3, 18, 31, 32, 33]
    tap_index = 1

    # Extract data
    train_data, test_data = extract_data(df_entire, taps_for_training, tap_index)

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
    X_test_scaled = scaler.transform(test_data[['x', 'y']])

    # Initialize dictionaries to store best results
    best_predictions = {}
    best_errors = {}
    best_model_instance = {}
    best_params = {}

    # Perform regression for each statistic and each model
    for stat in ['min_peak', 'mean', 'std']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None
        best_model = None
        best_param = None

        for model_name, model in models.items():
            # Perform grid search with cross-validation
            grid_search = GridSearchCV(model, param_grids[model_name], cv=3)
            grid_search.fit(X_train_scaled, y_train)

            # Use the best model found
            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            # Store the best parameters and errors for each model
            if model_name not in best_params:
                best_params[model_name] = {}
            best_params[model_name][stat] = {
                'params': grid_search.best_params_,
                'error': percentage_error,
                'pred': y_pred
            }

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name
                best_param = grid_search.best_params_

        best_predictions[stat] = best_pred
        best_errors[stat] = min_error
        best_model_instance[stat] = models[best_model_name]
        best_params[stat] = best_param

    # Store results for the current angle
    angle_results[angle] = {
        'predictions': best_predictions,
        'errors': best_errors,
        'models': best_model_instance,
        'params': best_params
    }

    # Print percentage errors for each statistic for the current angle
    print(f'\nResults for angle {angle}:')
    for stat in ['min_peak', 'mean', 'std']:
        print(f'{stat} - True value: {test_data[stat].values[0]}, Predicted value: {best_predictions[stat]}, Percentage error: {best_errors[stat]:.2f}%, Best model: {type(best_model_instance[stat]).__name__}, Parameters: {best_params[stat]}')

# Display the results for all angles in a tabular format
results_list = []
for angle, results in angle_results.items():
    for stat in ['min_peak', 'mean', 'std']:
        results_list.append({
            'Angle': angle,
            'Statistic': stat,
            'True Value': test_data[stat].values[0],
            'Predicted Value': results['predictions'][stat],
            'Percentage Error': results['errors'][stat],
            'Best Model': type(results['models'][stat]).__name__,
            'Parameters': results['params'][stat]
        })

results_df = pd.DataFrame(results_list)
print(results_df)

#### Predicting the Statistics of a Partially Unknown Time History

In [ ]:
# Predictions for a fixed wind angle and predicted tap

tap_index = 1
taps_for_training = [2, 16, 17, 3, 18, 31, 32, 33]

df_entire = df_entire_double_45

# Extract data for the specified taps (index -1 because it's zero-indexed)
train_data_first_block = pd.DataFrame({
    'tap': taps_for_training,
    'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
    'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
    'block': [1] * len(taps_for_training),
    'min_peak': [df_entire.iloc[0, tap - 1][0] for tap in taps_for_training],
    'mean': [df_entire.iloc[0, tap - 1][1] for tap in taps_for_training],
    'std': [df_entire.iloc[0, tap - 1][2] for tap in taps_for_training]
})

train_data_second_block = pd.DataFrame({
    'tap': taps_for_training,
    'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
    'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
    'block': [2] * len(taps_for_training),
    'min_peak': [df_entire.iloc[1, tap - 1][0] for tap in taps_for_training],
    'mean': [df_entire.iloc[1, tap - 1][1] for tap in taps_for_training],
    'std': [df_entire.iloc[1, tap - 1][2] for tap in taps_for_training]
})

# Include the first block of the test tap in the training set
tap_index_first_block = pd.DataFrame({
    'tap': [tap_index],
    'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
    'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
    'block': [1],
    'min_peak': [df_entire.iloc[0, tap_index-1][0]],
    'mean': [df_entire.iloc[0, tap_index-1][1]],
    'std': [df_entire.iloc[0, tap_index-1][2]]
})

train_data = pd.concat([train_data_first_block, train_data_second_block, tap_index_first_block], ignore_index=True)

# Extract data for the test tap (tap_index is 1-based, so we use tap_index-1 for 0-based indexing)
test_data = pd.DataFrame({
    'tap': [tap_index],
    'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
    'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
    'block': [2],
    'min_peak': [df_entire.iloc[1, tap_index-1][0]],  # Second block of tap 1
    'mean': [df_entire.iloc[1, tap_index-1][1]],
    'std': [df_entire.iloc[1, tap_index-1][2]]
})

# Extract training features and labels
X_train = train_data[['x', 'y', 'block']]
X_test = test_data[['x', 'y', 'block']]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNeighbors': KNeighborsRegressor(),
    'NeuralNetwork': MLPRegressor(max_iter=3000),
    'GaussianProcess': GaussianProcessRegressor()
}

# Define parameter grids for each model
param_grids = {
    'DecisionTree': {'max_depth': [None, 5, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'criterion': ['squared_error', 'friedman_mse', 'absolute_error']},
    'RandomForest': {'n_estimators': [25, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10, 20]},
    'GradientBoosting': {'n_estimators': [10, 25, 50, 100, 150], 'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2], 'max_depth': [2, 3, 5, 10]},
    'KNeighbors': {'n_neighbors': [2, 3, 5, len(taps_for_training)], 'weights': ['uniform', 'distance'], 'p': [1, 2]},
    'NeuralNetwork': {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)], 'alpha': [0.0001, 0.001, 0.01], 'solver': ['adam', 'lbfgs'], 'max_iter': [3000]},
    'GaussianProcess': {
        'kernel': [
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RationalQuadratic(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
        ],
        'alpha': [1e-2, 1e-5, 1e-10, 1e-12],
        'normalize_y': [False, True]
    }
}

# Initialize dictionaries to store predictions and errors
best_predictions = {}
best_errors = {}
best_model = {}
best_params = {}

# Perform regression for each statistic and each model
for stat in ['min_peak', 'mean', 'std']:
    y_train = train_data[stat]
    y_true = test_data[stat].values[0]

    min_error = float('inf')
    best_pred = None
    best_model_name = None
    best_model_params = None

    for model_name, model in models.items():
        grid_search = GridSearchCV(model, param_grids[model_name], cv=5)
        grid_search.fit(X_train_scaled, y_train)
        y_pred = grid_search.predict(X_test_scaled)[0]

        percentage_error = 100 * abs((y_true - y_pred) / y_true)

        if percentage_error < min_error:
            min_error = percentage_error
            best_pred = y_pred
            best_model_name = model_name
            best_model_params = grid_search.best_params_

    best_predictions[stat] = best_pred
    best_errors[stat] = min_error
    best_model[stat] = best_model_name
    best_params[stat] = best_model_params

# Calculate and display percentage errors
for stat in ['min_peak', 'mean', 'std']:
    print(f'{stat} - True value: {test_data[stat].values[0]}, Predicted value: {best_predictions[stat]}, Percentage error: {best_errors[stat]:.2f}%, Best model: {best_model[stat]}, Best parameters: {best_params[stat]}')

# Compute the average percentage error
average_error = sum(best_errors.values()) / len(best_errors)
print(f'Average Percentage Error: {average_error:.2f}%')

# Displaying the results in a tabular format
results_df = pd.DataFrame({
    'Statistic': ['min_peak', 'mean', 'std'],
    'True Value': [test_data[stat].values[0] for stat in ['min_peak', 'mean', 'std']],
    'Predicted Value': [best_predictions[stat] for stat in ['min_peak', 'mean', 'std']],
    'Percentage Error': [best_errors[stat] for stat in ['min_peak', 'mean', 'std']],
    'Best Model': [best_model[stat] for stat in ['min_peak', 'mean', 'std']],
    'Best Parameters': [best_params[stat] for stat in ['min_peak', 'mean', 'std']]
})

# Display the DataFrame using Pandas
print(results_df)


In [ ]:
# Predictions for a all wind angles and a specific predicted tap

# Define angles to iterate over
angles = [0, 15, 30, 45, 60, 75, 90]

# Define parameter grids for each model
param_grids = {
    'DecisionTree': {'max_depth': [None, 5, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'criterion': ['squared_error', 'friedman_mse', 'absolute_error']},
    'RandomForest': {'n_estimators': [25, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10, 20]},
    'GradientBoosting': {'n_estimators': [10, 25, 50, 100], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 10]},
    'KNeighbors': {'n_neighbors': [2, 3, 5], 'weights': ['uniform', 'distance'], 'p': [1, 2]},
    'NeuralNetwork': {'hidden_layer_sizes': [(50,), (50,50), (100,)], 'alpha': [0.0001, 0.001], 'solver': ['adam'], 'max_iter': [3000]},
    'GaussianProcess': {
        'kernel': [
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
            C(1.0, constant_value_bounds=(1e-2, 1e2)) * RationalQuadratic(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
        ],
        'alpha': [1e-2, 1e-5, 1e-10],
        'normalize_y': [False, True]
    }
}

# Initialize lists to store errors and best models for each angle
angle_errors = []
best_models_info = []

# Loop through each angle
for angle in angles:
    # Set the appropriate data frame for the given angle
    df_entire = globals()[f'df_entire_double_{angle}']

    tap_index = 1
    taps_for_training = [2, 16, 17, 3, 18, 31, 32, 33]

    tap_index = 61
    taps_for_training = [46, 47, 48, 62, 63, 76, 77, 78]

    tap_index = 81
    taps_for_training = [65, 66, 67, 80, 82, 95, 96, 97]

    # Extract data for the specified taps (index -1 because it's zero-indexed)
    train_data_first_block = pd.DataFrame({
        'tap': taps_for_training,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
        'block': [1] * len(taps_for_training),
        'min_peak': [df_entire.iloc[0, tap - 1][0] for tap in taps_for_training],
        'mean': [df_entire.iloc[0, tap - 1][1] for tap in taps_for_training],
        'std': [df_entire.iloc[0, tap - 1][2] for tap in taps_for_training]
    })

    train_data_second_block = pd.DataFrame({
        'tap': taps_for_training,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps_for_training],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps_for_training],
        'block': [2] * len(taps_for_training),
        'min_peak': [df_entire.iloc[1, tap - 1][0] for tap in taps_for_training],
        'mean': [df_entire.iloc[1, tap - 1][1] for tap in taps_for_training],
        'std': [df_entire.iloc[1, tap - 1][2] for tap in taps_for_training]
    })

    # Include the first block of the test tap in the training set
    tap_index_first_block = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'block': [1],
        'min_peak': [df_entire.iloc[0, tap_index-1][0]],
        'mean': [df_entire.iloc[0, tap_index-1][1]],
        'std': [df_entire.iloc[0, tap_index-1][2]]
    })

    train_data = pd.concat([train_data_first_block, train_data_second_block, tap_index_first_block], ignore_index=True)

    # Extract data for the test tap (tap_index is 1-based, so we use tap_index-1 for 0-based indexing)
    test_data = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'block': [2],
        'min_peak': [df_entire.iloc[1, tap_index-1][0]],  # Second block of tap 1
        'mean': [df_entire.iloc[1, tap_index-1][1]],
        'std': [df_entire.iloc[1, tap_index-1][2]]
    })

    # Extract training features and labels
    X_train = train_data[['x', 'y', 'block']]
    X_test = test_data[['x', 'y', 'block']]

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize models
    models = {
        'DecisionTree': DecisionTreeRegressor(),
        'RandomForest': RandomForestRegressor(),
        'GradientBoosting': GradientBoostingRegressor(),
        'KNeighbors': KNeighborsRegressor(),
        'NeuralNetwork': MLPRegressor(max_iter=3000),
        'GaussianProcess': GaussianProcessRegressor()
    }

    # Initialize dictionaries to store predictions and errors
    best_predictions = {}
    best_errors = {}
    best_model = {}
    best_params = {}

    # Perform regression for each statistic and each model
    for stat in ['min_peak', 'mean', 'std']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None
        best_model_params = None

        for model_name, model in models.items():
            grid_search = GridSearchCV(model, param_grids[model_name], cv=5)
            grid_search.fit(X_train_scaled, y_train)
            y_pred = grid_search.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name
                best_model_params = grid_search.best_params_

        best_predictions[stat] = best_pred
        best_errors[stat] = min_error
        best_model[stat] = best_model_name
        best_params[stat] = best_model_params

    # Store the errors and best model info for this angle
    angle_errors.append({
        'Angle': angle,
        'Min Peak Error': best_errors['min_peak'],
        'Mean Error': best_errors['mean'],
        'Std Error': best_errors['std']
    })
    
    best_models_info.append({
        'Angle': angle,
        'Min Peak Best Model': best_model['min_peak'],
        'Min Peak Best Params': best_params['min_peak'],
        'Mean Best Model': best_model['mean'],
        'Mean Best Params': best_params['mean'],
        'Std Best Model': best_model['std'],
        'Std Best Params': best_params['std']
    })

# Create DataFrames to display the errors and best model info for each angle
errors_df = pd.DataFrame(angle_errors)
best_models_df = pd.DataFrame(best_models_info)

# Display the DataFrames using Pandas
print(errors_df)
print(best_models_df)


#### Predicting the Statistics of a Missing Wind Angle Time History

In [ ]:
# Predictions for a missing wind angle and a specific predicted tap given the time histories for the other wind angles for that tap

# Combine the dataframes with the angle information
data_frames = [df_entire_0, df_entire_15, df_entire_30, df_entire_45, df_entire_60, df_entire_75, df_entire_90]
angles = [0, 15, 30, 45, 60, 75, 90]

tap_no = 81

# Create a DataFrame to store all results
all_results = []

# Loop through each angle to be excluded
for angle_excluded in angles:
    tap_data = pd.DataFrame({
        'angle': angles,
        'stat_1': [df.iloc[0, tap_no-1] for df in data_frames],
        'stat_2': [df.iloc[1, tap_no-1] for df in data_frames],
        'stat_3': [df.iloc[2, tap_no-1] for df in data_frames]
    })

    # Separate data into training (excluding the current angle) and testing (current angle)
    train_data = tap_data[tap_data['angle'] != angle_excluded]
    test_data = tap_data[tap_data['angle'] == angle_excluded]

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data[['angle']])
    X_test_scaled = scaler.transform(test_data[['angle']])

    # Define parameter grids for each model
    param_grids = {
        'DecisionTree': {'max_depth': [None, 5, 10, 25], 'min_samples_split': [2, 5, 10, 20], 'criterion': ['squared_error', 'friedman_mse', 'absolute_error']},
        'RandomForest': {'n_estimators': [25, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10, 20]},
        'GradientBoosting': {'n_estimators': [10, 25, 50, 100, 150], 'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2], 'max_depth': [2, 3, 5, 10]},
        'KNeighbors': {'n_neighbors': [2, 3, len(angles)-1], 'weights': ['uniform', 'distance'], 'p': [1, 2]},
        'NeuralNetwork': {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)], 'alpha': [0.0001, 0.001, 0.01], 'solver': ['adam', 'lbfgs'], 'max_iter': [3000]},
        'GaussianProcess': {
            'kernel': [
                C(1.0, constant_value_bounds=(1e-2, 1e2)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
                C(1.0, constant_value_bounds=(1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
                C(1.0, constant_value_bounds=(1e-2, 1e2)) * RationalQuadratic(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
            ],
            'alpha': [1e-2, 1e-5, 1e-10, 1e-12],
            'normalize_y': [False, True]
        }
    }


    models = {
        'DecisionTree': DecisionTreeRegressor(),
        'RandomForest': RandomForestRegressor(),
        'GradientBoosting': GradientBoostingRegressor(),
        'KNeighbors': KNeighborsRegressor(),
        'NeuralNetwork': MLPRegressor(),
        'GaussianProcess': GaussianProcessRegressor()
    }

    # Initialize dictionaries to store predictions, errors, and best parameters
    best_predictions = {}
    best_errors = {}
    best_model_instance = {}
    best_params = {}

    # Perform regression for each statistic and each model
    for stat in ['stat_1', 'stat_2', 'stat_3']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None
        best_model = None
        best_param = None

        for model_name, model in models.items():
            # Perform grid search with cross-validation
            grid_search = GridSearchCV(model, param_grids[model_name], cv=3)
            grid_search.fit(X_train_scaled, y_train)

            # Use the best model found
            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            # Store the best parameters and errors for each model
            if model_name not in best_params:
                best_params[model_name] = {}
            best_params[model_name][stat] = {
                'params': grid_search.best_params_,
                'error': percentage_error,
                'pred': y_pred
            }

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name
                best_param = grid_search.best_params_

        best_predictions[stat] = best_pred
        best_errors[stat] = min_error
        best_model_instance[stat] = models[best_model_name]

        # Append results to the overall results list
        all_results.append({
            'Excluded Angle': angle_excluded,
            'Statistic': stat,
            'True Value': y_true,
            'Predicted Value': best_pred,
            'Percentage Error': min_error,
            'Best Model': type(best_model_instance[stat]).__name__
        })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(all_results)

# Display the DataFrame using Pandas
print(results_df)

# Print the best parameters and errors for each model
for model_name in best_params:
    print(f'\nModel: {model_name}')
    for stat in ['stat_1', 'stat_2', 'stat_3']:
        print(f'  Statistic: {stat}')
        print(f'    Best Params: {best_params[model_name][stat]["params"]}')
        print(f'    Predicted Value: {best_params[model_name][stat]["pred"]}')
        print(f'    Percentage Error: {best_params[model_name][stat]["error"]:.2f}%')


#### Ranking the Importance of Training Taps

In [ ]:
# Ranking the importance of each individual training tap by computing the error percentage after removal of that pressure tap

# Function to compute the percentage errors and average error percentage for the removal of a specific tap
def compute_errors(taps_for_training, tap_index, df_entire, removed_tap):
    # Extract data for the specified taps excluding the removed tap
    train_taps = [tap for tap in taps_for_training if tap != removed_tap]

    train_data = pd.DataFrame({
        'tap': train_taps,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in train_taps],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in train_taps],
        'min_peak': [df_entire.iloc[0, tap - 1] for tap in train_taps],
        'mean': [df_entire.iloc[1, tap - 1] for tap in train_taps],
        'std': [df_entire.iloc[2, tap - 1] for tap in train_taps]
    })

    # Extract data for the test tap
    test_data = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'min_peak': [df_entire.iloc[0, tap_index-1]],
        'mean': [df_entire.iloc[1, tap_index-1]],
        'std': [df_entire.iloc[2, tap_index-1]]
    })

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
    X_test_scaled = scaler.transform(test_data[['x', 'y']])

    # Initialize models
    models = {
        'DecisionTree': DecisionTreeRegressor(),
        'RandomForest': RandomForestRegressor(n_estimators=100),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100),
        'KNeighbors': KNeighborsRegressor(n_neighbors=len(train_taps)),
        'NeuralNetwork': MLPRegressor(max_iter=3000),
        'GaussianProcess': GaussianProcessRegressor()
    }

    best_errors = {}

    # Perform regression for each statistic and each model
    for stat in ['min_peak', 'mean', 'std']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None

        for model_name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name

        best_errors[stat] = min_error

    average_error = np.mean(list(best_errors.values()))
    return average_error

# List of taps to use for training
taps_for_training = [2, 3, 4, 5, 16, 17, 18, 19, 20, 31, 32, 33, 34, 35, 46, 47, 48, 49, 50, 61, 62, 63, 64, 65]
tap_index = 1

df_entire = df_entire_45

# Dictionary to store the average error for each removed tap
tap_errors = {}

# Iterate over each training tap and compute the average error when it is removed
for tap in taps_for_training:
    average_error = compute_errors(taps_for_training, tap_index, df_entire, tap)
    tap_errors[tap] = average_error
    print(f'Tap {tap} - Average Error: {average_error:.2f}%')

# Rank the taps based on the average error
ranked_taps = sorted(tap_errors.items(), key=lambda x: x[1], reverse=True)

# Display the ranking
print("\nTap Importance Ranking (Higher average error percentage after removal means more important):")
for rank, (tap, avg_error) in enumerate(ranked_taps, 1):
    print(f'Rank {rank}: Tap {tap} - Average Error: {avg_error:.2f}%')

# Visualization part
x_coords, y_coords = zip(*roof_pressure_tap_coordinates)
x_limits = [0, 45]  # Adjust these limits as needed based on your data
y_limits = [0, 60]  # Adjust these limits as needed based on your data

# Extract the ranked training taps
sorted_trained_taps = [tap for tap, _ in ranked_taps]

# Filter out the coordinates for sorted training taps
filtered_x_coords_train = [x_coords[tap - 1] for tap in sorted_trained_taps]
filtered_y_coords_train = [y_coords[tap - 1] for tap in sorted_trained_taps]
filtered_errors = [error for _, error in ranked_taps]

# Normalize the errors for coloring
norm = plt.Normalize(min(filtered_errors), max(filtered_errors))
colors = plt.cm.viridis(norm(filtered_errors))

# Create the 2D plot with the sorted training taps using a heatmap
plt.figure(figsize=(10, 8))

# Plot sorted training taps with colors indicating importance
plt.scatter(filtered_x_coords_train, filtered_y_coords_train, color=colors, label='Training Taps', s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index - 1][1]
plt.scatter(predicted_x, predicted_y, color='red', label='Predicted Tap', s=100)

# Create a colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
sm.set_array([])
plt.colorbar(sm, label='Importance')

plt.xlim(x_limits)
plt.ylim(y_limits)
plt.gca().set_aspect('equal', adjustable='box')  # Set equal scaling on the axes
plt.title('Importance of Each Training Tap')
plt.grid(True)
# Adding labels
plt.xlabel('X Location (inches)')
plt.ylabel('Y Location (inches)')
# Adding legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)
plt.show()


In [ ]:
# Ranking the importance of each individual training tap by computing the error percentage after removal of that pressure tap (for multiple predicted taps separately)

# Function to compute the percentage errors and average error percentage for the removal of a specific tap
def compute_errors(taps_for_training, tap_index, df_entire, removed_tap):
    # Extract data for the specified taps excluding the removed tap
    train_taps = [tap for tap in taps_for_training if tap != removed_tap]

    train_data = pd.DataFrame({
        'tap': train_taps,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in train_taps],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in train_taps],
        'min_peak': [df_entire.iloc[0, tap - 1] for tap in train_taps],
        'mean': [df_entire.iloc[1, tap - 1] for tap in train_taps],
        'std': [df_entire.iloc[2, tap - 1] for tap in train_taps]
    })

    # Extract data for the test tap
    test_data = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'min_peak': [df_entire.iloc[0, tap_index-1]],
        'mean': [df_entire.iloc[1, tap_index-1]],
        'std': [df_entire.iloc[2, tap_index-1]]
    })

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
    X_test_scaled = scaler.transform(test_data[['x', 'y']])

    # Initialize models
    models = {
        'DecisionTree': DecisionTreeRegressor(),
        'RandomForest': RandomForestRegressor(n_estimators=100),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100),
        'KNeighbors': KNeighborsRegressor(n_neighbors=len(train_taps)),
        'NeuralNetwork': MLPRegressor(max_iter=3000),
        'GaussianProcess': GaussianProcessRegressor()
    }

    best_errors = {}

    # Perform regression for each statistic and each model
    for stat in ['min_peak', 'mean', 'std']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None

        for model_name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name

        best_errors[stat] = min_error

    average_error = np.mean(list(best_errors.values()))
    return average_error

# Taps configurations
taps_configurations = {
    1: ([2, 3, 4, 5, 16, 17, 18, 19, 20, 31, 32, 33, 34, 35, 46, 47, 48, 49, 50, 61, 62, 63, 64, 65], df_entire_45),
    61: ([46, 47, 62, 76, 77, 31, 32, 33, 48, 63, 78, 91, 92, 93, 16, 17, 18, 19, 34, 49, 64, 79, 94, 106, 107, 108, 109], df_entire_0),
    81: ([65, 66, 67, 80, 82, 95, 96, 97, 49, 50, 51, 52, 53, 64, 68, 79, 83, 94, 98, 109, 110, 111, 112, 113], df_entire_0)
}



# Prepare the figure
fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharey=True)

# Process each configuration
for ax, (tap_index, (taps_for_training, df_entire)) in zip(axes, taps_configurations.items()):
    tap_errors = {}

    # Compute the average error for each tap
    for tap in taps_for_training:
        average_error = compute_errors(taps_for_training, tap_index, df_entire, tap)
        tap_errors[tap] = average_error
        print(f'Tap {tap} - Average Error: {average_error:.2f}%')

    # Rank the taps based on the average error
    ranked_taps = sorted(tap_errors.items(), key=lambda x: x[1], reverse=True)

    # Visualization part
    x_coords, y_coords = zip(*roof_pressure_tap_coordinates)
    x_limits = [0, 45]  # Adjust these limits as needed based on your data
    y_limits = [0, 60]  # Adjust these limits as needed based on your data

    # Extract the ranked training taps
    sorted_trained_taps = [tap for tap, _ in ranked_taps]

    # Filter out the coordinates for sorted training taps
    filtered_x_coords_train = [x_coords[tap - 1] for tap in sorted_trained_taps]
    filtered_y_coords_train = [y_coords[tap - 1] for tap in sorted_trained_taps]
    filtered_errors = [error for _, error in ranked_taps]

    # Normalize the errors for coloring
    norm = plt.Normalize(min(filtered_errors), max(filtered_errors))
    colors = plt.cm.viridis(norm(filtered_errors))

    # Plot sorted training taps with colors indicating importance
    sc = ax.scatter(filtered_x_coords_train, filtered_y_coords_train, color=colors, s=100)

    # Plot the predicted tap in red
    predicted_x = roof_pressure_tap_coordinates[tap_index - 1][0]
    predicted_y = roof_pressure_tap_coordinates[tap_index - 1][1]
    ax.scatter(predicted_x, predicted_y, color='red', label='Predicted Tap', s=100)

    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)
    ax.set_aspect('equal', adjustable='box')  # Set equal scaling on the axes
    ax.set_title(f'Tap {tap_index}')
    ax.grid(True)
    ax.set_xlabel('X Location (inches)')
    if ax == axes[0]:
        ax.set_ylabel('Y Location (inches)')

# Create a colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=axes, orientation='vertical', fraction=0.02, pad=0.04)
cbar.set_label('Importance')

# Adding legend once for all
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3)

plt.show()


In [ ]:
# Ranking the importance of each layer of training taps by computing the error percentage when only that layer is used in training

# Function to compute the percentage errors and average error percentage for the removal of a specific tap
def compute_errors(taps_for_training, tap_index, df_entire, removed_tap=None, model_type='all'):
    if removed_tap is not None:
        train_taps = [tap for tap in taps_for_training if tap != removed_tap]
    else:
        train_taps = taps_for_training

    train_data = pd.DataFrame({
        'tap': train_taps,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in train_taps],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in train_taps],
        'min_peak': [df_entire.iloc[0, tap - 1] for tap in train_taps],
        'mean': [df_entire.iloc[1, tap - 1] for tap in train_taps],
        'std': [df_entire.iloc[2, tap - 1] for tap in train_taps]
    })

    test_data = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'min_peak': [df_entire.iloc[0, tap_index-1]],
        'mean': [df_entire.iloc[1, tap_index-1]],
        'std': [df_entire.iloc[2, tap_index-1]]
    })

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
    X_test_scaled = scaler.transform(test_data[['x', 'y']])

    if model_type == 'all':
        # Initialize models
        models = {
            'DecisionTree': DecisionTreeRegressor(),
            'RandomForest': RandomForestRegressor(n_estimators=100),
            'GradientBoosting': GradientBoostingRegressor(n_estimators=100),
            'KNeighbors': KNeighborsRegressor(n_neighbors=len(train_taps)),
            'NeuralNetwork': MLPRegressor(max_iter=3000),
            'GaussianProcess': GaussianProcessRegressor()
        }
    elif model_type == 'DecisionTree':
        models = {'DecisionTree': DecisionTreeRegressor()}
    elif model_type == 'KNeighbors':
        models = {'KNeighbors': KNeighborsRegressor(n_neighbors=len(train_taps))}
    elif model_type == 'RandomForest':
        models = {'RandomForest': RandomForestRegressor(n_estimators=100)}
    elif model_type == 'GradientBoosting':
        models = {'GradientBoosting': GradientBoostingRegressor(n_estimators=100)}
    elif model_type == 'NeuralNetwork':
        models = {'NeuralNetwork': MLPRegressor(max_iter=3000)}
    elif model_type == 'GaussianProcess':
        models = {'GaussianProcess': GaussianProcessRegressor()}

    best_errors = {}

    # Perform regression for each statistic and each model
    for stat in ['min_peak', 'mean', 'std']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None

        for model_name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name

        best_errors[stat] = min_error

    average_error = np.mean(list(best_errors.values()))
    return average_error, best_model_name


# Function to compute the errors for each layer separately
def compute_layer_errors_separately(first_layer, second_layer, third_layer, tap_index, df_entire):
    layer_errors = {}
    layers = [first_layer, second_layer, third_layer]
    layer_labels = ['Layer 1', 'Layer 2', 'Layer 3']

    for layer, label in zip(layers, layer_labels):
        layer_errors[label], _ = compute_errors(layer, tap_index, df_entire)
        print(f'{label} Error: {layer_errors[label]:.2f}%')

    return layer_errors

# Taps configurations for tap 1
taps_for_training_1 = [2, 3, 4, 5, 16, 17, 18, 19, 20, 31, 32, 33, 34, 35, 46, 47, 48, 49, 50, 61, 62, 63, 64, 65]
tap_index_1 = 1
df_entire_1 = df_entire_45

# Taps configurations for tap 61
first_layer_61 = [46, 47, 62, 76, 77]
second_layer_61 = [31, 32, 33, 48, 63, 78, 91, 92, 93]
third_layer_61 = [16, 17, 18, 19, 34, 49, 64, 79, 94, 106, 107, 108, 109]
tap_index_61 = 61
df_entire_61 = df_entire_0

# Taps configurations for tap 81
first_layer_81 = [65, 66, 67, 80, 82, 95, 96, 97]
second_layer_81 = [49, 50, 51, 52, 53, 64, 68, 79, 83, 94, 98, 109, 110, 111, 112, 113]
third_layer_81 = [33, 34, 35, 36, 37, 38, 39, 54, 69, 84, 99, 114, 48, 63, 78, 93, 108, 123, 124, 125, 126, 127, 128, 129]
tap_index_81 = 81
df_entire_81 = df_entire_90


# Compute errors for each tap for tap 1
tap_errors_1 = {}
for tap in taps_for_training_1:
    tap_errors_1[tap], _ = compute_errors(taps_for_training_1, tap_index_1, df_entire_1, removed_tap=tap)
    print(f'Tap {tap} - Error: {tap_errors_1[tap]:.2f}%')
tap_errors_1 = sorted(tap_errors_1.items(), key=lambda x: x[1], reverse=True)

# Compute errors for each layer separately for tap 61
layer_errors_61 = compute_layer_errors_separately(first_layer_61, second_layer_61, third_layer_61, tap_index_61, df_entire_61)

# Compute errors for each layer separately for tap 81
layer_errors_81 = compute_layer_errors_separately(first_layer_81, second_layer_81, third_layer_81, tap_index_81, df_entire_81)

# Prepare the figure
fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharey=True)

# Plot for tap 1
ax = axes[0]
x_coords, y_coords = zip(*roof_pressure_tap_coordinates)
sorted_taps_1, errors_1 = zip(*tap_errors_1)
norm = plt.Normalize(min(errors_1), max(errors_1))
colors_1 = plt.cm.viridis(norm(errors_1))

filtered_x_coords_1 = [x_coords[tap - 1] for tap in sorted_taps_1]
filtered_y_coords_1 = [y_coords[tap - 1] for tap in sorted_taps_1]
ax.scatter(filtered_x_coords_1, filtered_y_coords_1, color=colors_1, s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index_1 - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index_1 - 1][1]
ax.scatter(predicted_x, predicted_y, color='red', s=100)

ax.set_xlim(0, 45)
ax.set_ylim(0, 60)
ax.set_aspect('equal', adjustable='box')
ax.set_title('Tap 1')
ax.grid(True)
ax.set_xlabel('X Location (inches)')
ax.set_ylabel('Y Location (inches)')

# Plot for tap 61
ax = axes[1]
layer_colors_61 = [layer_errors_61['Layer 1'], layer_errors_61['Layer 2'], layer_errors_61['Layer 3']]
layer_labels_61 = ['Layer 1', 'Layer 2', 'Layer 3']
layer_taps_61 = [first_layer_61, second_layer_61, third_layer_61]
norm = plt.Normalize(min(layer_colors_61), max(layer_colors_61))
colors_61 = plt.cm.viridis(norm(layer_colors_61))

for taps, color, label in zip(layer_taps_61, colors_61, layer_labels_61):
    filtered_x_coords = [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps]
    filtered_y_coords = [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps]
    ax.scatter(filtered_x_coords, filtered_y_coords, color=color, label=label, s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index_61 - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index_61 - 1][1]
ax.scatter(predicted_x,

 predicted_y, color='red', s=100)

ax.set_xlim(0, 45)
ax.set_ylim(0, 60)
ax.set_aspect('equal', adjustable='box')
ax.set_title('Tap 61')
ax.grid(True)
ax.set_xlabel('X Location (inches)')

# Plot for tap 81
ax = axes[2]
layer_colors_81 = [layer_errors_81['Layer 1'], layer_errors_81['Layer 2'], layer_errors_81['Layer 3']]
layer_labels_81 = ['Layer 1', 'Layer 2', 'Layer 3']
layer_taps_81 = [first_layer_81, second_layer_81, third_layer_81]
colors_81 = plt.cm.viridis(norm(layer_colors_81))

for taps, color, label in zip(layer_taps_81, colors_81, layer_labels_81):
    filtered_x_coords = [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps]
    filtered_y_coords = [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps]
    ax.scatter(filtered_x_coords, filtered_y_coords, color=color, label=label, s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index_81 - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index_81 - 1][1]
ax.scatter(predicted_x, predicted_y, color='red', s=100)

ax.set_xlim(0, 45)
ax.set_ylim(0, 60)
ax.set_aspect('equal', adjustable='box')
ax.set_title('Tap 81')
ax.grid(True)
ax.set_xlabel('X Location (inches)')

# Create a colorbar without numbers
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=axes, orientation='vertical', fraction=0.02, pad=0.04)
cbar.set_label('Importance')
cbar.ax.yaxis.set_major_locator(plt.NullLocator())

# Adding legend once for all
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3)

plt.show()


In [ ]:
# Ranking the importance of each training tap layers by computing the error percentage when each layer is additively included in the set of training taps

# Function to compute the percentage errors and average error percentage for the removal of a specific tap
def compute_errors(taps_for_training, tap_index, df_entire, removed_tap=None, model_type='all'):
    if removed_tap is not None:
        train_taps = [tap for tap in taps_for_training if tap != removed_tap]
    else:
        train_taps = taps_for_training

    train_data = pd.DataFrame({
        'tap': train_taps,
        'x': [roof_pressure_tap_coordinates[tap - 1][0] for tap in train_taps],
        'y': [roof_pressure_tap_coordinates[tap - 1][1] for tap in train_taps],
        'min_peak': [df_entire.iloc[0, tap - 1] for tap in train_taps],
        'mean': [df_entire.iloc[1, tap - 1] for tap in train_taps],
        'std': [df_entire.iloc[2, tap - 1] for tap in train_taps]
    })

    test_data = pd.DataFrame({
        'tap': [tap_index],
        'x': [roof_pressure_tap_coordinates[tap_index-1][0]],
        'y': [roof_pressure_tap_coordinates[tap_index-1][1]],
        'min_peak': [df_entire.iloc[0, tap_index-1]],
        'mean': [df_entire.iloc[1, tap_index-1]],
        'std': [df_entire.iloc[2, tap_index-1]]
    })

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train_data[['x', 'y']])
    X_test_scaled = scaler.transform(test_data[['x', 'y']])

    if model_type == 'all':
        # Initialize models
        models = {
            'DecisionTree': DecisionTreeRegressor(),
            'RandomForest': RandomForestRegressor(n_estimators=100),
            'GradientBoosting': GradientBoostingRegressor(n_estimators=100),
            'KNeighbors': KNeighborsRegressor(n_neighbors=len(train_taps)),
            'NeuralNetwork': MLPRegressor(max_iter=3000),
            'GaussianProcess': GaussianProcessRegressor()
        }
    elif model_type == 'DecisionTree':
        models = {'DecisionTree': DecisionTreeRegressor()}
    elif model_type == 'KNeighbors':
        models = {'KNeighbors': KNeighborsRegressor(n_neighbors=len(train_taps))}
    elif model_type == 'RandomForest':
        models = {'RandomForest': RandomForestRegressor(n_estimators=100)}
    elif model_type == 'GradientBoosting':
        models = {'GradientBoosting': GradientBoostingRegressor(n_estimators=100)}
    elif model_type == 'NeuralNetwork':
        models = {'NeuralNetwork': MLPRegressor(max_iter=3000)}
    elif model_type == 'GaussianProcess':
        models = {'GaussianProcess': GaussianProcessRegressor()}

    best_errors = {}

    # Perform regression for each statistic and each model
    for stat in ['min_peak', 'mean', 'std']:
        y_train = train_data[stat]
        y_true = test_data[stat].values[0]

        min_error = float('inf')
        best_pred = None
        best_model_name = None

        for model_name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)[0]

            percentage_error = 100 * abs((y_true - y_pred) / y_true)

            if percentage_error < min_error:
                min_error = percentage_error
                best_pred = y_pred
                best_model_name = model_name

        best_errors[stat] = min_error

    average_error = np.mean(list(best_errors.values()))
    return average_error, best_model_name

# Function to compute the errors for each layer additively
def compute_layer_errors_additively(first_layer, second_layer, third_layer, tap_index, df_entire):
    layer_errors = {}
    combined_layers = [first_layer, first_layer + second_layer, first_layer + second_layer + third_layer]
    layer_labels = ['Layer 1', 'Layer 1+2', 'Layer 1+2+3']

    for taps, label in zip(combined_layers, layer_labels):
        layer_errors[label], _ = compute_errors(taps, tap_index, df_entire, model_type='GaussianProcess')
        print(f'{label} Error: {layer_errors[label]:.2f}%')

    return layer_errors

# Taps configurations for tap 1
first_layer_taps_1 = [2, 16, 17]
second_layer_taps_1 = [3, 18, 31, 32, 33]
third_layer_taps_1 = [4, 19, 34, 46, 47, 48, 49]
tap_index_1 = 1
df_entire_1 = df_entire_90

# Taps configurations for tap 61
first_layer_61 = [46, 47, 62, 76, 77]
second_layer_61 = [31, 32, 33, 48, 63, 78, 91, 92, 93]
third_layer_61 = [16, 17, 18, 19, 34, 49, 64, 79, 94, 106, 107, 108, 109]
tap_index_61 = 61
df_entire_61 = df_entire_90

# Taps configurations for tap 81
first_layer_81 = [65, 66, 67, 80, 82, 95, 96, 97]
second_layer_81 = [49, 50, 51, 52, 53, 64, 68, 79, 83, 94, 98, 109, 110, 111, 112, 113]
third_layer_81 = [33, 34, 35, 36, 37, 38, 39, 54, 69, 84, 99, 114, 48, 63, 78, 93, 108, 123, 124, 125, 126, 127, 128, 129]
tap_index_81 = 81
df_entire_81 = df_entire_90

# Compute errors for each layer additively for tap 1
layer_errors_1 = compute_layer_errors_additively(first_layer_taps_1, second_layer_taps_1, third_layer_taps_1, tap_index_1, df_entire_1)

# Compute errors for each layer additively for tap 61
layer_errors_61 = compute_layer_errors_additively(first_layer_61, second_layer_61, third_layer_61, tap_index_61, df_entire_61)

# Compute errors for each layer additively for tap 81
layer_errors_81 = compute_layer_errors_additively(first_layer_81, second_layer_81, third_layer_81, tap_index_81, df_entire_81)

# Invert the importance for tap 61 and tap 81 (higher error means lower importance)
max_error_61 = max(layer_errors_61.values())
min_error_61 = min(layer_errors_61.values())
layer_errors_61 = {k: (max_error_61 - v + min_error_61) for k, v in layer_errors_61.items()}

max_error_81 = max(layer_errors_81.values())
min_error_81 = min(layer_errors_81.values())
layer_errors_81 = {k: (max_error_81 - v + min_error_81) for k, v in layer_errors_81.items()}

# Prepare the figure
fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharey=True)

# Plot for tap 1
ax = axes[0]
x_coords, y_coords = zip(*roof_pressure_tap_coordinates)
layer_colors_1 = [layer_errors_1['Layer 1'], layer_errors_1['Layer 1+2'], layer_errors_1['Layer 1+2+3']]
layer_labels_1 = ['Layer 1', 'Layer 1+2', 'Layer 1+2+3']
layer_taps_1 = [first_layer_taps_1, first_layer_taps_1 + second_layer_taps_1, first_layer_taps_1 + second_layer_taps_1 + third_layer_taps_1]
norm = plt.Normalize(min(layer_colors_1), max(layer_colors_1))
colors_1 = plt.cm.viridis(norm(layer_colors_1))

for taps, color, label in zip(layer_taps_1, colors_1, layer_labels_1):
    filtered_x_coords = [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps]
    filtered_y_coords = [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps]
    ax.scatter(filtered_x_coords, filtered_y_coords, color=color, label=label, s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index_1 - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index_1 - 1][1]
ax.scatter(predicted_x, predicted_y, color='red', s=100)

ax.set_xlim(0, 45)
ax.set_ylim(0, 60)
ax.set_aspect('equal', adjustable='box')
ax.set_title('Tap 1')
ax.grid(True)
ax.set_xlabel('X Location (inches)')
ax.set_ylabel('Y Location (inches)')

# Plot for tap 61
ax = axes[1]
layer_colors_61 = [layer_errors_61['Layer 1'], layer_errors_61['Layer 1+2'], layer_errors_61['Layer 1+2+3']]
layer_labels_61 = ['Layer 1', 'Layer 1+2', 'Layer 1+2+3']
layer_taps_61 = [first_layer_61, first_layer_61 + second_layer_61, first_layer_61 + second_layer_61 + third_layer_61]
norm = plt.Normalize(min(layer_colors_61), max(layer_colors_61))
colors_61 = plt.cm.viridis(norm(layer_colors_61))

for taps, color, label in zip(layer_taps_61, colors_61, layer_labels_61):
    filtered_x_coords = [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps]
    filtered_y_coords = [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps]
    ax.scatter(filtered_x_coords, filtered_y_coords, color=color, label=label, s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index_61 - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index_61 - 1][1]
ax.scatter(predicted_x, predicted_y, color='red', s=100)

ax.set_xlim(0, 45)
ax.set_ylim(0, 60)
ax.set_aspect('equal', adjustable='box')
ax.set_title('Tap 61')
ax.grid(True)
ax.set_xlabel('X Location (inches)')

# Plot for tap 81
ax = axes[2]
layer_colors_81 = [layer_errors_81['Layer 1'], layer_errors_81['Layer 1+2'], layer_errors_81['Layer 1+2+3']]
layer_labels_81 = ['Layer 1', 'Layer 1+2', 'Layer 1+2+3']
layer_taps_81 = [first_layer_81, first_layer_81 + second_layer_81, first_layer_81 + second_layer_81 + third_layer_81]
colors_81 = plt.cm.viridis(norm(layer_colors_81))

for taps, color, label in zip(layer_taps_81, colors_81, layer_labels_81):
    filtered_x_coords = [roof_pressure_tap_coordinates[tap - 1][0] for tap in taps]
    filtered_y_coords = [roof_pressure_tap_coordinates[tap - 1][1] for tap in taps]
    ax.scatter(filtered_x_coords, filtered_y_coords, color=color, label=label, s=100)

# Plot the predicted tap in red
predicted_x = roof_pressure_tap_coordinates[tap_index_81 - 1][0]
predicted_y = roof_pressure_tap_coordinates[tap_index_81 - 1][1]
ax.scatter(predicted_x, predicted_y, color='red', s=100)

ax.set_xlim(0, 45)
ax.set_ylim(0, 60)
ax.set_aspect('equal', adjustable='box')
ax.set_title('Tap 81')
ax.grid(True)
ax.set_xlabel('X Location (inches)')

# Create a colorbar without numbers
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=axes, orientation='vertical', fraction=0.02, pad=0.04)
cbar.set_label('Importance')
cbar.ax.yaxis.set_major_locator(plt.NullLocator())

# Adding legend once for all
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3)

plt.show()



### Regression on Pressure Coefficients

#### Predicting the Statistics of a Completely Unknown Time History

#### Predicting the Statistics of a Partially Unknown Time History